# 📚  Exercise Session - Week 7
**Main Topics**: Generation, Decoding, Beam-search, Sampling, NLG Eval Metrics


Welcome to Week 7 of CS-552 Modern NLP's exercise sessions!
The aim of this exercise is to get yourself acquainted with generation algorithms that are used with language models. 
We demonstrate the different generation algorithms with a fine-tuned version of the [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf) model.

1. [Story Generator Model:](#1-story-generator-model)
    - Load a GPT-2 model that's only been pretrained
    - Load a GPT-2 model finetuned on the BookCorpus dataset
2. [Decoding Algorithms](#2-decoding-algorithms)
    1. [Greedy Search](#21-greedy-search)
    2. [Beam Search](#22-beam-search)
    3. [Decoding Takeaways](#23-decoding-takeaways)
3. [Sampling Algorithms:](#3-nucleus-sampling)
    1. [Random Sampling](#31-random-sampling)
    2. [Top-$p$ Sampling](#32-top--nucleus)
    3. [Sampling Takeaways](#33-sampling-takeaways)
4. [Automatic NLG Evaluation Metrics - the curious case of BLEU:](#4-automatic-nlg-evaluation-metrics---the-curious-case-of-bleu)
    1. [Pitfall 1: Correct Paraphrases](#41-pitfall-1-accurate-paraphrases)
    2. [Pitfall 2: Senseless Repetitions](#42-pitfall-2-senseless-repetitions)
    3. [Other Problems](#43-other-problems-with-bleu)
5. [Conclusion](#5-conclusion)

> **By the end of the session you will be able to:**
> - ✅  Generate stories with Language Models
> - ✅  Use decoder algorithms such as greedy and beam search with huggingface and understand their pitfalls
> - ✅  Use sampling algorithms such as top-k and top-k sampling with huggingface and understand their pitfalls 
> - ✅  Understand the BLEU NLG evaluation metric and recognize its pitfalls

<div style="padding:8px 0 8px 15px;border-left:3px solid orange;">

### ⚠️ **Note: Cells are pre-run. You don't need to execute any cells, but you are welcome to. It will download the finetuned GPT2 model which might take around 1.5-3 minutes depending on your internet connection.** ⚠️
So if you would like to, you can skip the setup section!

### Setup
- Colab: If you are using Colab (which has Python version 3.9.16) then download the following packages by creating a code cell below:
```
!pip install -U ipykernel
!pip install -U ipywidgets
!pip install -U setuptools 
!pip install -U wheel
!pip install -U transformers datasets evaluate
!pip install -U torch torchmetrics torchvision
```

- Local: If you are not using Colab, create an environment with Python version 3.9.16 and then install the packages in this exercise session's `requirements.txt` (not the one from the beginning of the course!) with `pip install -r requirements.txt`.

### Acknowledgements
We are very grateful to [Patrick von Platen](https://huggingface.co/patrickvonplaten) for writing a [blogpost on different generation techniques](https://huggingface.co/blog/how-to-generate). Our notebook takes its examples, images, and wordings. 
Moreover, we would like to thank [Google Cloud Translation's evaluation documentation](https://cloud.google.com/translate/automl/docs/evaluate#:~:text=BLEU%20(BiLingual%20Evaluation%20Understudy)%20is,of%20high%20quality%20reference%20translations), which we have used for the evaluation section. We encourage you to check these resources out!

The model we primarily use is the 125M parameter [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf). The [finetuned story generation version on huggingface](https://huggingface.co/pranavpsv/genre-story-generator-v2) was finetuned using the [BookCorpus dataset](https://paperswithcode.com/dataset/bookcorpus) by getting the different genres per book and prefixing the inputs with the relevant genre.

</div>


---

## 1) Story Generator Model

First, we import the pretrained GPT-2 and a finetuned version of it with [huggingface](https://huggingface.co/)'s [transformers package](https://huggingface.co/docs/transformers/index). The latter is fine-tuned on sample of BookCorpus dataset for short story generation. This model has been fine-tuned to first take the `<BOS>` token and then a token that depicts the genre we desire to generate such as `<adventure>`. Possible genres this model handles and their respective tokens are:

- Romance `<romance>`
- Adventure `<adventure>`
- Mystery & detective `<mystery-&-detective`
- Fantasy `<fantasy>`
- Humor & comedy `<humor-&-comedy>`
- Paranormal `<paranormal>`
- Science fiction `<science-fiction>`

In [13]:
# 1) Import the relevant packages
import torch
import evaluate
from transformers import GPT2LMHeadModel, GPT2Tokenizer


# 2) Load the relevant models and their respective tokenizers
# NOTE: You could also try gpt2-medium for the pretrained model! Although generation will be a bit slower.
pretrained_model_id = "gpt2"
story_model_id = "pranavpsv/genre-story-generator-v2"
#
pretrained_tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_id)
story_tokenizer = GPT2Tokenizer.from_pretrained(story_model_id)
#
pretrained_model = GPT2LMHeadModel.from_pretrained(pretrained_model_id , pad_token_id=pretrained_tokenizer.eos_token_id) 
story_model = GPT2LMHeadModel.from_pretrained(story_model_id, pad_token_id=story_tokenizer.eos_token_id)
#
pretrained_model.eval()
story_model.eval()

# 3) A display function to compare the pretrained and story-finetuned GPT-2 model generations
def display_outputs(pretrained_output, story_output, multiple_output=False):
    if not multiple_output:
        print(80 * '-')
        print("Pretrained model output:")
        print(80 * '-')
        print(pretrained_tokenizer.decode(pretrained_output[0], skip_special_tokens=True))
        print(80 * '-')
        print("Story model output:")
        print(80 * '-')
        print(story_tokenizer.decode(story_output[0], skip_special_tokens=True))
    else:
        print(80 * '-')
        print("Pretrained model output:")
        print(80 * '-')
        for i, beam_output in enumerate(pretrained_output):
            if i > 0:
                print(20 * '-')
            print("{}: {}".format(i + 1, pretrained_tokenizer.decode(beam_output, skip_special_tokens=True)))
        
        print(80 * '-')
        print("Story model output:")
        print(80 * '-')
        for i, beam_output in enumerate(story_output):
            if i > 0:
                print(20 * '-')
            print("{}: {}".format(i + 1, story_tokenizer.decode(beam_output, skip_special_tokens=True)))


## 2) Decoding Algorithms

### 2.1) Greedy Search

Greedy search is the first type of decoding algorithm one can imagine: we select the next token as the one with the highest probability.
Given the following image provided by Patrick von Platen, the path that a greedy search would choose is depicted by the red path.

![Greedy Search](./figs/greedy_search.png)

Given the word **The**, the algorithm picks *greedily* the highest probability token **nice**, and then **woman**. Note that the generated token sequence has therefore a probability of:

$0.5 \times 0.4 = 0.2$.

Now let's generate a sequence after inputting the model the sequence **"Bella couldn't take it anymore. Edward was gone for 3 months now and"** inspired by second movie and book of Twilight. For the story generation version we will add the prefix token `<romance>`. 

Note that in *transformers*, we can generate with autoregressive models simply by calling the model's *generate* function and giving it tokenized inputs as shown below:

In [14]:
# 1) Tokenize the text with which we condition the generation
pretrained_input_ids = pretrained_tokenizer.encode("Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')
story_input_ids = story_tokenizer.encode("<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')

# 2) Generate text until the output length (which includes the context length) 
#    reaches max_length
greedy_pretrained_output = pretrained_model.generate(pretrained_input_ids, max_length=100)
greedy_story_output = story_model.generate(story_input_ids, max_length=100)

# 3) Decode the output back into readable strings
display_outputs(greedy_pretrained_output, greedy_story_output)


--------------------------------------------------------------------------------
Pretrained model output:
--------------------------------------------------------------------------------
Bella couldn't take it anymore. Edward was gone for 3 months now and Bella was still in the hospital.

"I'm so sorry, Bella," Edward said. "I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so
--------------------------------------------------------------------------------
Story model output:
--------------------------------------------------------------------------------
<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and Bella was still in love with him. Bella wanted to be with Edward but she couldn't. Bella wanted to be with Edward but she couldn't. Bella wanted to be with Edward but she couldn't. Bella wanted to be with Edward but she couldn't. Bella w

**Q:**
- **Given the generated output above, what is a serious problem you find about the greedy search algorithm?**
- **Why do you think this might be happening?**
- **Is there a difference between the pretrained model and the story finetuned model with respect to these pitfalls?**

*A: TODO - your answer here!*

### 2.2) Beam Search

Instead of considering a single best next token, the beam search algorithm tries to respond to problems with greedy search by considering the next *num_beam* amount of tokens that have the highest joint probability. Consider the initial token sequence probability tree illustration with *num_beams*=2 (since *num_beams*=1 would be greedy decoding!):

![Beam Search](./figs/beam_search.png)

- In the first step t=0, **The nice** seems to have the highest probability, so the model chooses **The**
- In the next time step t=1, we notice that the joint probability of **The dog has** ($0.4 \times 0.9 = 0.36$) is higher than **The nice woman**'s ($0.5 \times 0.4 = 0.2$)

$\implies$ therefore the model eventualy picks the red branch sequence **The dog has**

Let's generate what beam search would give for the same input sentence **"Bella couldn't take it anymore. Edward was gone for 3 months now and"**. We also do *early_stopping*=True, which means that we stop generating if the token is the end-of-sentence [EOS] token.

In [15]:
# 1) Tokenize the text with which we condition the generation
pretrained_input_ids = pretrained_tokenizer.encode("Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')
story_input_ids = story_tokenizer.encode("<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')

# 2) Generate text until the output length (which includes the context length) 
#    reaches max_length or early stopping
beam_pretrained_output = pretrained_model.generate(
    pretrained_input_ids, 
    max_length=100, 
    num_beams=4, # set num_beams=4, default num_beams=1
    early_stopping=True
)
beam_story_output = story_model.generate(
    story_input_ids, 
    max_length=100, 
    num_beams=4, # set num_beams=4, default num_beams=1
    early_stopping=True
)

# 3) Decode the output back into readable strings
display_outputs(beam_pretrained_output, beam_story_output)

--------------------------------------------------------------------------------
Pretrained model output:
--------------------------------------------------------------------------------
Bella couldn't take it anymore. Edward was gone for 3 months now and Bella had no idea what was going on.

"I'm sorry, Bella," Edward said.

"I'm sorry," Bella said.

"I'm sorry," Edward said.

"I'm sorry," Bella said.

"I'm sorry," Edward said.

"I'm sorry," Bella said.

"I'm sorry," Edward said.

"
--------------------------------------------------------------------------------
Story model output:
--------------------------------------------------------------------------------
<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and Bella couldn't take it anymore. Bella wanted to be with Edward, but she couldn't take it anymore. Bella wanted to be with Edward, but she couldn't take it anymore. Bella wanted to be with Edward, but she couldn't take it anymore. Bella wanted to be w

**Q:**
- **What is slightly different between the greedy search and the beam search outputs?**
- **Given the generated output above, what is a serious problem you find about the beam search algorithm?**
- **What is a possible solution?**
- **Is there still a difference between the pretrained model and the story finetuned model with respect to these pitfalls?**

*A: TODO - your answer here!*

A possible solution proposed by [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304) and [Klein et al. (2017)](https://arxiv.org/abs/1701.02810) is to penalize the model for generating a sequence of grams (another word for words) of $n$-grams ($n$ instances of grams). One way is to make sure that no $n$-gram appears $k$ times by manually setting the probability of next words that could create an already seen $n$-gram to 0.

Let's try it out by setting *no_repeat_ngram_size*=2:

In [16]:
# 1) Tokenize the text with which we condition the generation
pretrained_input_ids = pretrained_tokenizer.encode("Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')
story_input_ids = story_tokenizer.encode("<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')

# 2) Generate text until the output length (which includes the context length) 
#    reaches max_length or early stopping
beam_pretrained_output = pretrained_model.generate(
    pretrained_input_ids, 
    max_length=100, 
    num_beams=4, 
    no_repeat_ngram_size=2, # set no_repeat_ngram_size=2
    early_stopping=True
)
beam_story_output = story_model.generate(
    story_input_ids, 
    max_length=100, 
    num_beams=4, 
    no_repeat_ngram_size=2, # set no_repeat_ngram_size=2
    early_stopping=True
)

# 3) Decode the output back into readable strings
display_outputs(beam_pretrained_output, beam_story_output)

--------------------------------------------------------------------------------
Pretrained model output:
--------------------------------------------------------------------------------
Bella couldn't take it anymore. Edward was gone for 3 months now and Bella had no idea what was going on.

"I don't know what to do," Bella said. "I just want to go back to school. I'm not going to be able to take care of my family anymore."
--------------------------------------------------------------------------------
Story model output:
--------------------------------------------------------------------------------
<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and Bella wanted to go back to her old life with Edward. Bella decided to leave Edward and move back in with her aunt and uncle. But Bella's aunt was not happy with the way Bella was living. She wanted Bella to stay in the house, but Bella didn't want to be a part of the family.  Bella and Edward had a lot of fun

Yay! It looks like most problematic repetitiveness is avoided. However, note that this an extreme approach. What if São Paulo or New York had to be mentioned in a generated article and we penalized the 2-gram city names? The algorithm won't re-use the bigram, and the text could look unnatural.

Another important property of generation is variety. For an application such as a writing aid, we might want to see different generated sentences, and pick the one we like the most. Given that there could be some subsequences that have close probabilities, can we get different outputs from beam-search? :

In [17]:
# 1) Tokenize the text with which we condition the generation
pretrained_input_ids = pretrained_tokenizer.encode("Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')
story_input_ids = story_tokenizer.encode("<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')

# 2) Generate text until the output length (which includes the context length) 
#    reaches max_length or early stopping
beam_pretrained_output = pretrained_model.generate(
    pretrained_input_ids, 
    max_length=100, 
    num_beams=4, 
    no_repeat_ngram_size=2, 
    early_stopping=True,
    num_return_sequences=3 # set num_return_sequences = 3
)
beam_story_output = story_model.generate(
    story_input_ids, 
    max_length=100, 
    num_beams=4, 
    no_repeat_ngram_size=2, 
    early_stopping=True,
    num_return_sequences=3 # set num_return_sequences = 3
)

# 3) Decode the output back into readable strings
display_outputs(beam_pretrained_output, beam_story_output, multiple_output=True)

--------------------------------------------------------------------------------
Pretrained model output:
--------------------------------------------------------------------------------
1: Bella couldn't take it anymore. Edward was gone for 3 months now and Bella had no idea what was going on.

"I don't know what to do," Bella said. "I just want to go back to school. I'm not going to be able to take care of my family anymore."
--------------------
2: Bella couldn't take it anymore. Edward was gone for 3 months now and Bella had no idea what was going on.

"I don't know what to do," Bella said. "I just want to go back to school. I'm not going to be able to take care of my family. It's been a long time since I've been here, and I can't do anything about it. But I know that if I do, I'll be back in the world.
--------------------
3: Bella couldn't take it anymore. Edward was gone for 3 months now and Bella had no idea what was going on.

"I don't know what to do," Bella said. "I just wan

**Q:**
- **What is the variation mostly related to?**
- **Given the results do you think this algorithm can be applied as a writing aid?**

*A: TODO - your answer here!*

### 2.3) Decoding Takeaways

As you can see for most generations, the most likely subsequence is unique and decoding algorithms can be too predictable. This is mainly because of two reasons:

- **Generation length:** Beam search works when tasks have predictable length (ex: machine translation or summarization) ([Murray et al. 2018](https://arxiv.org/abs/1808.10006), [Yang et al. 2018](https://arxiv.org/abs/1808.09582)). It's not great for open-ended generation where the desired output length can vary (ex: dialog).

- **Suprisal effect:** [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751) show that high quality human language does not follow a distribution of high probability next words. It's desirable for the generated text to be surprising.

Therefore, we need to consider stochastic generation algorithms. Will Bella ever get over Edward? Let's sample to see what happens!

## 3) Sampling Algorithms

### 3.1) Random Sampling

Sampling means randomly picking the next word $w_t$ from ​its conditional probability distribution $P(w_{t}|w_{1:t-1})$. Let's picture what this can be like with the prior probability trees we have seen:

![Beam Search](./figs/sampling_search.png)

Each dot here represents a timestamp in decoding. To choose the second token **car** we randomly pick a token among [**nice**, **dog**, **car**]. The model then gives next possible tokens' probabilities and the generation algorithm *randomly* chooses **drives**, which happens to be the top-probability token.

However, as you can imagine, this may not create very coherent sentences. Let's try it out with GPT-2!

In [18]:
# 1) Tokenize the text with which we condition the generation
pretrained_input_ids = pretrained_tokenizer.encode("Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')
story_input_ids = story_tokenizer.encode("<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')

# 2) Generate text until the output length (which includes the context length) 
#    reaches max_length or early stopping
# NOTE: setting the seed so the output doesn't always change at every run, 
#       but you can comment the next line out to see how much it can vary.
seed = 84
torch.manual_seed(84)
sample_pretrained_output = pretrained_model.generate(
    pretrained_input_ids, 
    do_sample=True, 
    max_length=100, 
    top_k=0
)
torch.manual_seed(84)
sample_story_output = story_model.generate(
    story_input_ids, 
    do_sample=True, 
    max_length=100, 
    top_k=0
)

# 3) Decode the output back into readable strings
display_outputs(sample_pretrained_output, sample_story_output)

--------------------------------------------------------------------------------
Pretrained model output:
--------------------------------------------------------------------------------
Bella couldn't take it anymore. Edward was gone for 3 months now and she'd once again turned into a ghost.

Jessia tried to escape. Things had changed. Her vocation was free. She tried to stay there but died rather than spend time alone on the tired edge of her own life. Nothing. But now she had to, even though she thought it came down to the nostalgia she had for her old life.

When there was a problem she wouldn't have
--------------------------------------------------------------------------------
Story model output:
--------------------------------------------------------------------------------
<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and she was seeing a kind of dark older man with aged eyes. Bella don't understand him anymore. Instead of calling out, she picks u

As expected the model repeats subsequences much less, but the output is incoherent due to the way we are randomly sampling from the **complete** output distribution.

**Q:**
- **What are some other possible ways that you can do sampling such that the content isn't completely incoherent?**
- **Think about algorithms seen in class. What do you think is best for this problem?**

*A: TODO - your answer here!*

### 3.2) Top-$p$ (nucleus) 

In class, we went over top-$p$ sampling (or also commonly referred to as nucleus sampling). The "thresholding" aspect of this algorithm is the cumulative probability mass, which is distributed among the top-$p$ answers that the algorithm can randomly choose from. This way the number of best $k$ tokens dynamically changes according to the model's confidence. Let's see how we can implement it with the *transformers* package. There are 2 types of arguments that we can pass to the model's *generate* function: 
1. *top_k* : the top $k$ highest probability tokens kept for sampling 
2. *top_p* : the cumulative probability mass from which the most likely words are sampled, if set to float < 1, only the smallest set of most probable tokens with probabilities that add up to $p$ **or higher** are kept for sampling.
    - **Or higher** because if no single token falls under this cumulative probability mass, for example, one token has a probability mass higher than *top_p*, then the default in huggingface is to greedily sample the top-scoring token.
    - However, you can imagine setting a minimum *top_k* such that if no token is in this cumulative probability mass, and getting further creative with such edge cases!

We set *top_k*=0 to only demonstrate nucleus sampling with *top_p*=0.7.

In [19]:
# 1) Tokenize the text with which we condition the generation
pretrained_input_ids = pretrained_tokenizer.encode("Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')
story_input_ids = story_tokenizer.encode("<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')

# 2) Generate text until the output length (which includes the context length) 
#    reaches max_length or early stopping
# NOTE: setting the seed so the output doesn't always change at every run, 
#       but you can comment the next line out to see how much it can vary.
seed = 84
torch.manual_seed(84)
sample_pretrained_output = pretrained_model.generate(
    pretrained_input_ids, 
    do_sample=True, 
    max_length=100, 
    top_p=0.7, # sample only from tokens falling in the top cumulative 70% probability
    top_k=0 # deactivate top_k sampling
)
torch.manual_seed(84)
sample_story_output = story_model.generate(
    story_input_ids, 
    do_sample=True, 
    max_length=100, 
    top_p=0.7, # sample only from tokens falling in the top cumulative 70% probability
    top_k=0 # deactivate top_k sampling
)

# 3) Decode the output back into readable strings
display_outputs(sample_pretrained_output, sample_story_output)

--------------------------------------------------------------------------------
Pretrained model output:
--------------------------------------------------------------------------------
Bella couldn't take it anymore. Edward was gone for 3 months now and she'd lost a lot of her feelings.

He'd tried to reassure her by saying that he'd had a few encounters with the Ruby, but that was just a tip of the iceberg.

"I don't know how you'd handle this, Edward," she said, tilting her head to look at him. "It's really like a nightmare, Edward."

"It's really like
--------------------------------------------------------------------------------
Story model output:
--------------------------------------------------------------------------------
<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and she was seeing a new guy. Bella wanted to be with him but she couldn't because she was afraid of him. When Bella is down with her new boyfriend Simon, her mother decides to get

The coherence is somewhat better with respect to the context given! But what about variance?

In [20]:
# 1) Tokenize the text with which we condition the generation
pretrained_input_ids = pretrained_tokenizer.encode("Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')
story_input_ids = story_tokenizer.encode("<romance> Bella couldn't take it anymore. Edward was gone for 3 months now and", return_tensors='pt')

# 2) Generate text until the output length (which includes the context length) 
#    reaches max_length or early stopping
# NOTE: setting the seed so the output doesn't always change at every run, 
#       but you can comment the next line out to see how much it can vary.
seed = 84
torch.manual_seed(84)
sample_pretrained_output = pretrained_model.generate(
    pretrained_input_ids, 
    do_sample=True, 
    max_length=100, 
    top_p=0.7, # sample only from tokens falling in the top cumulative 70% probability
    top_k=0, # deactivate top_k sampling
    num_return_sequences=3 # set num_return_sequences = 3
)
torch.manual_seed(84)
sample_story_output = story_model.generate(
    story_input_ids, 
    do_sample=True, 
    max_length=100, 
    top_p=0.7, # sample only from tokens falling in the top cumulative 70% probability
    top_k=0, # deactivate top_k sampling
    num_return_sequences=3 # set num_return_sequences = 3
)

# 3) Decode the output back into readable strings
display_outputs(sample_pretrained_output, sample_story_output, multiple_output=True)

--------------------------------------------------------------------------------
Pretrained model output:
--------------------------------------------------------------------------------
1: Bella couldn't take it anymore. Edward was gone for 3 months now and she'd lost a lot of her feelings.

He'd tried to reassure her by saying that he'd had a few encounters with the Ruby, but that was just a tip of the iceberg.

"I don't know how you'd handle this, Edward," she said, tilting her head to look at him. "It's really like a nightmare, Edward."

"It's really like
--------------------
2: Bella couldn't take it anymore. Edward was gone for 3 months now and he wanted to know how he was going to do things for her.

So she asked her dad for his help in finding Edward. They agreed and he told Bella how to keep her safe.

He told her how to get"))esa home to stop going to the vet and have his wife return to her.

She told him that Edward had already been kidnapped and that he had to go home
-----

As we somewhat expected, the outputs are much more diverse than those of deterministic decoding algorithms. And it looks like Bella does not get a happy ending if *num_return_sequences*=3 🥲 even if all three endings are unique in their own way.

### 3.3) Sampling Takeaways

Sampling can be really finicky according to the input text we condition our generation with and the top-$k$ or top-$p$ hyperparameter options we choose. 

However, it offers a wider variety of outputs rather than decoding algorithms like greedy and beam search.

## 4) Automatic NLG Evaluation Metrics - the curious case of BLEU

### Motivation
In this section, we will take a close look on how to evaluate generated text quality. In translation or other sequence-to-sequence tasks, we do not have a single way to get a *correct* answer. For example, consider a model that outputs the text:

> The cat is fluffly.

and the correct *reference* text we have is:

> The cat has fluffy hair.

These two sentences look like paraphases but are not exact matches. Therefore these tasks require a more flexible evaluation in which we check for matching subsequences of $n$-grams. For this reason, we will take a close look to the popular BLEU score (BiLingual Evaluation Understudy).

### What is BLEU?
BLEU is a metric proposed by [Papineni et al (2002)](https://aclanthology.org/P02-1040.pdf) for automatic evaluation of machine translation. The BLEU score is number between 0 and 1 that compares the similarity of a predicted *candidate* text to a set of correct *reference* texts, where 1 means that the generated text is close to the reference and is of "high quality" and 0 is the opposite. BLEU was originally used for machine translation but can also be applied for any text generation task (ex: summarization). Note that often in practice the BLEU score is reported on a scale of 1 to 100 for granularity, but it's not an accuracy metric!

So how do we calculate BLEU? In short, it has two terms:

1. **$n$-gram overlap calculation:** BLEU calculates the n-gram overlap counts of how many *candidate* unigrams, bigrams, trigrams, and four-grams ($n$=1,...,4) match their $n$-gram counterpart in the *reference* sequence. **This can therefore be considered as a precision metric.** While unigrams might be capturing lexical level similarity, we can imagine that longer $n$-gram matching captures higher-level syntactic constituent similarity.

2. **brevity penalty:** Since BLEU doesn't capture recall with the overlap calculation, we augment it with a brevity penalty. This penalizes generations that are too short compared to a reference length.

For more details and an exact equation, checkout Google Cloud Translation's [evaluation documentation](https://cloud.google.com/translate/automl/docs/evaluate#:~:text=BLEU%20(BiLingual%20Evaluation%20Understudy)%20is,of%20high%20quality%20reference%20translations.).

Despite being automatic and easy to use, BLEU has many problems with it, depending on where and how it's used. Let's find out what issues there may with it!

### 4.1) Pitfall 1: Accurate paraphrases

BLEU measures gram-by-gram similarity between the *candidate* and the *reference* text. This means that, correct paraphrases can hurt the score. Let's see how this might happen when we want to translate:
- input: **Le chat s'est assis sur le tapis**
- candidate/prediction/output: **The feline layed over the rug**
- reference: **The cat sat on the mat**

In [21]:
# NOTE: the huggingface evaluate package provides a BLEU score that returns 
#       precision at all 4 levels and how much brevity penalty was applied
bleu = evaluate.load("bleu")

# 1) Accurate paraphrase
predictions = ["the feline layed over the rug"]
references = [
    ["the cat sat on the mat"],
]
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.0,
 'precisions': [0.3333333333333333, 0.0, 0.0, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 6,
 'reference_length': 6}

As you can see while the paraphased sentence is close to the reference, it gets only a slight score on BLEU-1 (leftmost in precisions), and none of BLEU-4 (rightmost in precisions) as the four-gram overlap is none. On the other hand, let's take a look at a semantically incorrect but bag-of-words-wise close sentences.

In [22]:
# 2) Removed stopwords
predictions = ["cat sat mat"]
references = [
    ["the cat sat on the mat"],
]
results = bleu.compute(predictions=predictions, references=references)
print(results)

# 3) switched "mat" and "cat"
predictions = ["the mat sat on the cat"]
references = [
    ["the cat sat on the mat"],
]
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.0, 'precisions': [1.0, 0.5, 0.0, 0.0], 'brevity_penalty': 0.36787944117144233, 'length_ratio': 0.5, 'translation_length': 3, 'reference_length': 6}


{'bleu': 0.0,
 'precisions': [1.0, 0.8, 0.25, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 6,
 'reference_length': 6}

Every gram that is in the reference text is also in the prediction. Therefore, BLEU-1 gets full score and BLEU-3 gets a reasonable precision on these examples. However, due to the fact that there are no four-gram overlaps, the overall BLEU score is 0.0.

### 4.2) Pitfall 2: Senseless repetitions

Are there any ways we could get the four-gram overlap without coherent mapping among the prediction and the reference?

- input: **Le chat s'est assis sur le tapis**
- prediction: 
    - **The cat sat on the cat, is on the cat** 
    - or **The cat sat on the cat, the mat sat on the mat**
- reference: **The cat sat on the mat**

In [23]:
predictions = ["the cat sat on the cat, is on the cat"]
references = [
    ["the cat sat on the mat"],
]
results = bleu.compute(predictions=predictions, references=references)
print(results)

predictions = ["the cat sat on the cat, the mat sat on the mat"]
references = [
    ["the cat sat on the mat"],
]
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.35084396956386854, 'precisions': [0.45454545454545453, 0.4, 0.3333333333333333, 0.25], 'brevity_penalty': 1.0, 'length_ratio': 1.8333333333333333, 'translation_length': 11, 'reference_length': 6}


{'bleu': 0.38058030016749456,
 'precisions': [0.46153846153846156,
  0.4166666666666667,
  0.36363636363636365,
  0.3],
 'brevity_penalty': 1.0,
 'length_ratio': 2.1666666666666665,
 'translation_length': 13,
 'reference_length': 6}

Now this is the first time we got a BLEU score that is above 0. As you can see, none of the translations actually make sense, it's simply because we can find a matching four-gram like "the cat sat on" ... "sat on the mat", and other possibilities. Despite all grams showing up, the BLEU-1 score is reduced due to a lack of brevity.

### 4.3) Other problems with BLEU

There are some important properties of BLEU that we need to consider when deciding to use it. For example:
1. BLEU performs particularly bad when tested on individual *reference* sentences, or when the length of the generation isn't consistent across a corpus.
2. At least one matching 4-gram is required to get a BLEU score > 0.
3. This score is also very dependent on the sort of tokenization used to get the $n$-grams.

## 5) Conclusion

We found out that greedy and beam search often result in repetitive patterns where the generation loops on subsequences.
However, there are penalization and sampling approaches that help us get coherent generation without the repetitive subsequences.

We also went over a popular NLG evaluation metric and saw how tough it can be to evaluate a task where the output is allowed to be diverse sequences. 
While BLEU is one option, there are many more metrics out there. Often in practise we have to use all metrics at the same time to have the big picture. 
Otherwise, as we have seen with BLEU, every metric has its own shortage and cannot give us a good understanding on how our model is behaving compared to a given large corpus of high quality references.

### What else can I do?

Checkout the huggingface [transformers text generation documentation](https://huggingface.co/docs/transformers/main_classes/text_generation) to see what other models you can generate with. There are many more sampling approaches being offered these days, and is a hot topic in NLP. If you want to read a bit more on new methods, checkout this paper on [Locally Typical Sampling](https://arxiv.org/abs/2202.00666) (a generation option that is now available with the *transformers* `generate` function).

You can also browse through their selection of [metrics](https://huggingface.co/docs/datasets/how_to_metrics#compute-scores) or the torch package's [metrics](https://torchmetrics.readthedocs.io/en/stable/all-metrics.html#) to find more NLG metrics.